In [ ]:
!pip install -q aitextgen
from aitextgen import aitextgen
import pandas as pd
from aitextgen.TokenDataset import TokenDataset
import json

### **Trent på total Arxiv**

In [ ]:
def get_clean_authors(authors):
    r = []
    for a in authors:
        r.append(" ".join(a).strip())
    return r

articles = []
category = 'nucl-ex'
with open("../input/arxiv/arxiv-metadata-oai-snapshot.json", "r") as f:
    for i,l in enumerate(f):
        #if i%100000==0:
            #print(i)
        d = json.loads(l)
        if category in d['categories'].split(' '):
            d['clean_authors'] = get_clean_authors(d['authors_parsed']) 
            articles.append(d)

articles_df = pd.DataFrame().from_records(articles)

In [ ]:
!pip3 install pylatexenc -q

In [ ]:
from pylatexenc.latex2text import LatexNodes2Text

In [ ]:
LatexNodes2Text().latex_to_text(articles_df['abstract'][2]).replace('\n', ' ').strip()

In [ ]:
%%time
# LaTex to UTF
clean_abstract = []
clean_title = []
for i,a in articles_df.iterrows():
    if i%100==0:
            print(i)
            
    # Clean abstract
    try:
        clean_abstract.append(LatexNodes2Text().latex_to_text(a['abstract']).replace('\n', ' ').strip()) 
    except:
        clean_abstract.append(a['abstract'].replace('\n', ' ').strip())
articles_df['clean_abstracts'] = clean_abstract
articles_df['clean_title'] = clean_title

In [ ]:
list_arxiv = list(articles_df.abstract)

count = 0
for abstract in list_arxiv:
    if "climate change" in abstract:
        count += 1
    elif "Climate change" in abstract:
        count += 1
    elif "global warming" in abstract:
        count += 1
    elif "Global warming" in abstract:
        count += 1



print(count, len(list_arxiv))
print(list_arxiv[2])

In [ ]:
training_data_arxiv = TokenDataset(texts=list_arxiv)

In [ ]:
ai_arxiv = aitextgen(model_name="gpt-small", to_gpu=True)

In [ ]:
%%time
ai_arxiv.train(training_data_arxiv, 
        line_by_line=True,
        from_cache=False,
        num_steps=100,
        learning_rate=1e-4,
        fp16=False,
        batch_size=1
        )

In [ ]:
ai_arxiv.generate(n=3,prompt='Climate change is', max_length=100)

### **Trent på fake news**

In [ ]:
df_fake_news = pd.read_csv("../input/fakenewsnet/BuzzFeed_fake_news_content.csv")

In [ ]:
list_fake_news = list(df_fake_news.text)
training_data_fn = TokenDataset(texts=list_fake_news)

In [ ]:
ai_fn = aitextgen(model_name="gpt-small", to_gpu=True)

In [ ]:
%%time
ai_fn.train(training_data_fn, 
        line_by_line=True,
        from_cache=False,
        num_steps=100,
        learning_rate=1e-4,
        fp16=False,
        batch_size=1
        )

In [ ]:
ai_fn.generate(n=3,prompt='Climate change is', max_length=100)

### **Trent på Arxiv-papers-2010-2020**

In [ ]:
df_arxiv_2010 = pd.read_json("../input/arxiv-papers-2010-2020/arXiv_title_abstract_20200809_2011_2020.json")

In [ ]:
list_summary = list(df_arxiv_2010.abstract)

count = 0
for summary in list_summary:
    if "climate change" in summary:
        count += 1
    elif "Climate change" in summary:
        count += 1
    elif "global warming" in summary:
        count += 1
    elif "Global warming" in summary:
        count += 1

print(count, len(list_summary))
print(list_summary[2:10])


In [ ]:
training_data_arxiv_2010 = TokenDataset(texts=list_summary)

In [ ]:
ai_arx_2010 = aitextgen(model_name="gpt-small", to_gpu=True)

In [ ]:
%%time
ai_arx_2010.train(training_data_arxiv_2010, 
        line_by_line=True,
        from_cache=False,
        num_steps=100,
        learning_rate=1e-4,
        fp16=False,
        batch_size=1
        )

In [ ]:
ai_arx_2010.generate(n=3,prompt='Climate change is', max_length=100)

### ****Trained on twitter-set****

In [ ]:
df1 = pd.read_csv("../input/twitter-climate-change-sentiment-dataset/twitter_sentiment_data.csv")

In [ ]:
twitter_message = []
for i in range(len(df1)):
    tweet = df1.message[i]
    if '@' in tweet:
        tweet = " ".join(filter(lambda x:x[0]!='@', tweet.split()))
    if 'RT' in tweet:
        tweet = " ".join(filter(lambda x:x[0:2]!='RT' , tweet.split()))
    if 'https' in tweet:
        tweet = " ".join(filter(lambda x:x[0:6]!='https' , tweet.split()))
    twitter_message.append(tweet)

In [ ]:
training_data_twitter = TokenDataset(texts=twitter_message)

In [ ]:
ai_twitter = aitextgen(model_name="gpt-small", to_gpu=True)

In [ ]:
ai_twitter.train(training_data_twitter, 
        line_by_line=True,
        from_cache=False,
        num_steps=100,
        learning_rate=1e-4,
        fp16=False,
        batch_size=1
        )

In [ ]:
ai_twitter.generate(n=3,prompt='Climate change is', max_length=100)

### **Trent på alle datasett over**

In [ ]:
ai_all = aitextgen(model_name="gpt-small", to_gpu=True)

In [ ]:
%%time

ai_all.train(training_data_arxiv, 
        line_by_line=True,
        from_cache=False,
        num_steps=100,
        learning_rate=1e-4,
        fp16=False,
        batch_size=1
        )

ai_all.train(training_data_arxiv_2010, 
        line_by_line=True,
        from_cache=False,
        num_steps=100,
        learning_rate=1e-4,
        fp16=False,
        batch_size=1
        )

ai_all.train(training_data_fn, 
        line_by_line=True,
        from_cache=False,
        num_steps=100,
        learning_rate=1e-4,
        fp16=False,
        batch_size=1
        )

ai_all.train(training_data_twitter, 
        line_by_line=True,
        from_cache=False,
        num_steps=100,
        learning_rate=1e-4,
        fp16=False,
        batch_size=1
        )

In [ ]:
ai_all.generate(n=3,prompt='Climate change is', max_length=100)

### **Trained on total arxiv**

In [ ]:
df_arxiv = pd.read_json("../input/arxivdataset/arxivData.json")

In [ ]:
# Viser at datasettet ikke er spesielt relevant
list_summary = list(df_arxiv[df_arxiv.summary==df_arxiv.summary].summary)

list_climate_summary = []
count = 0
for summary in list_summary:
    if "climate change" in summary:
        count += 1
        list_relevant_summary.append(summary)
    elif "Climate change" in summary:
        count += 1
        list_relevant_summary.append(summary)
    elif "global warming" in summary:
        count += 1
        list_relevant_summary.append(summary)
    elif "Global warming" in summary:
        count += 1
        list_relevant_summary.append(summary)

print(count)
#training_data = TokenDataset(texts=list_summary)

In [ ]:
ai = aitextgen(model_name="gpt-small", to_gpu=True)

In [ ]:
ai.train(training_data, 
        line_by_line=True,
        from_cache=False,
        num_steps=20,
        learning_rate=1e-4,
        fp16=False,
        batch_size=1
        )

#generate_every=20,

In [ ]:
ai.generate(n=3,prompt='Climate change is', max_length=100)

### **Trained on 5 articles**

In [ ]:
articles_file = open("../input/5-cc-articles/combined.txt", "r")
articles = articles_file.read()
articles_list = articles.split("\n")
print(articles_list[3]) #OBS, fjern tomrom
articles_list = [name for name in articles_list if name.strip()]
print(articles_list[1])
#training_data = TokenDataset(texts=articles_list)

In [ ]:
print(len(articles_list))

In [ ]:
ai = aitextgen(model_name="gpt-small", to_gpu=True)

In [ ]:
ai.train(training_data, 
        line_by_line=True,
        from_cache=False,
        num_steps=100,
        generate_every=10,
        learning_rate=1e-4,
        fp16=False,
        batch_size=1
        )

In [ ]:
ai.generate(n=5,prompt='Climate change is', max_length=100)

### **Trained on many articles**

In [ ]:
articles_file = open("../input/many-cc-articles/articles.txt", "r")
articles = articles_file.read()
articles_list = articles.split("\n")
print(articles_list[3]) #OBS, fjern tomrom
articles_list = [name for name in articles_list if name.strip()]
print(articles_list[10])
#training_data = TokenDataset(texts=articles_list)

### **Not-trained model** (saved as files)

In [ ]:
ai = aitextgen(model_name="gpt-small", to_gpu=True)

In [ ]:
ai.generate(n=3, prompt="Climate change is", max_length=100)

In [ ]:
ai.generate_to_file(n=3, prompt="Trump is president because", max_length=100)

In [ ]:
ai.generate_to_file(n=3, prompt="Climate change is", max_length=100)
ai.generate_to_file(n=3, prompt="The coronavirus is", max_length=100)
ai.generate_to_file(n=3, prompt="What is Black Lives Matter?", max_length=100)
ai.generate_to_file(n=3, prompt="What is white supremacy?", max_length=100)